In [32]:
%use dataframe
%use coroutines
import java.time.LocalDateTime

var df = DataFrame.readCSV("data/pre_fault_dump.csv")
df.schema()

untitled: Int
txid: Int
blockid: Int
status: Double
creator_msp_id: String
validation_code: String
chaincode_proposal_input: Double
chaincode: String
key: String
access_type: String
version_block: Double?
version_tx: Double?
mvcc_cause: Int


In [33]:
import kotlinx.coroutines.sync.Mutex
import kotlinx.coroutines.sync.withLock
import kotlinx.coroutines.Dispatchers.Default

val start = System.currentTimeMillis()
val mutex = Mutex()

for(i in 0 until(df.rowsCount())){
    val row = df[i]
    if(row["validation_code"]=="MVCC_READ_CONFLICT" && row["access_type"]=="READ"){
        var j = i
        val blockId = row.blockid
        val key = row["key"]
        val versionB = row.version_block
        if(versionB!=null){
            CoroutineScope(Default).launch{
                while(df.blockid[j]>versionB || j==0){
                    j--
                    val rowj = df[j]
                    if(rowj.key==key && rowj.validation_code=="VALID" && rowj.access_type=="WRITE"){
                        mutex.withLock{
                            df = df.update{ mvcc_cause }.at(j).with { it+1 }
                        }
                    break
                    }
                } 
            }  
        }
    }
}
val end = System.currentTimeMillis()
print(end-start)
df

137

... showing only top 20 of 92883 rows DataFrame: rowsCount = 92883, columnsCount = 13

In [45]:
%use lets-plot
%use lets-plot-dataframe
import jetbrains.letsPlot.*
import jetbrains.letsPlot.geom.*
import jetbrains.letsPlot.stat.*

var idk = df.groupBy{ key }.sum("mvccs_caused"){ mvcc_cause }
idk = idk.sortByDesc("mvccs_caused").filter { "mvccs_caused"<Int>() > 0 }
idk
// val data = mapOf("keys" to idk["key"],
//                 "mvccs caused" to idk["mvccs_caused"])
// var p = letsPlot(data)
// p+= geomBar(width=0.1)
// p

// df = df.sortByDesc{ mvccs_caused }

DataFrame: rowsCount = 3, columnsCount = 2